In [117]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
import os

# Fecha de referencia (equivalente a Sys.Date() en el código R)
REFERENCE_DATE = pd.Timestamp('2024-01-01')

# Cargar los datasets
data_path = Path('../data/raw')  # Ajusta esta ruta según tu estructura de directorios
transactions = pd.read_csv(data_path / 'transactions.csv')
demographics = pd.read_csv(data_path / 'demographics.csv')
products = pd.read_csv(data_path / 'products.csv')

# Convertir fechas a formato datetime
transactions['date'] = pd.to_datetime(transactions['date'])
products['contract_date'] = pd.to_datetime(products['contract_date'])

# 1. Preparar el dataset de demografía (age_range_sturges)
breaks = np.linspace(18, 70, 9)  # 9 puntos de corte para 8 segmentos
labels = ["18–24", "25–31", "32–38", "39–45", "46–52", "53–59", "60–66", "67–70"]
demographics['age_range_sturges'] = pd.cut(
    demographics['age'], 
    bins=breaks, 
    labels=labels, 
    right=True, 
    include_lowest=True
)

# 2. Preparar el dataset de productos
products_mod = products.copy()
products_mod['product_type'] = products_mod['product_type'].replace('investment_account', 'investment')

# Crear variables binarias para cada producto
products_wide = pd.pivot_table(
    products_mod,
    index='user_id',
    columns='product_type',
    values='contract_date',
    aggfunc='count',
    fill_value=0
).reset_index()

# Convertir a indicadores binarios (0/1)
product_types = ['checking_account', 'savings_account', 'credit_card', 'insurance', 'investment']
for col in product_types:
    if col not in products_wide.columns:
        products_wide[col] = 0
    products_wide[col] = (products_wide[col] > 0).astype(int)

# Extraer fechas de primer/segundo producto
product_dates = (
    products_mod
    .sort_values(['user_id', 'contract_date'])
    .groupby('user_id')
    .apply(lambda x: pd.Series({
        'primer_producto': x['product_type'].iloc[0],
        'fecha_primer_producto': x['contract_date'].iloc[0],
        'segundo_producto': x['product_type'].iloc[1] if len(x) >= 2 else np.nan,
        'fecha_segundo_producto': x['contract_date'].iloc[1] if len(x) >= 2 else pd.NaT,
    }))
    .reset_index()
)

# Calcular días entre productos y antigüedad del cliente
product_dates['dias_entre_productos'] = (
    product_dates['fecha_segundo_producto'] - product_dates['fecha_primer_producto']
).dt.days

product_dates['antiguedad_cliente'] = (
    REFERENCE_DATE - product_dates['fecha_primer_producto']
).dt.days

# Join de products_wide y product_dates
products_final = (
    product_dates
    .merge(products_wide, on='user_id', how='left')
)

# Calcular número de productos
products_final['numero_productos'] = products_final[product_types].sum(axis=1)

# Generar combinación de productos
def get_product_combination(row):
    products = []
    for product in product_types:
        if row[product] == 1:
            products.append(product)
    
    if not products:
        return "sin_productos"
    
    return " + ".join(products)

products_final['combinacion_productos'] = products_final.apply(get_product_combination, axis=1)

# Reemplazar con las combinaciones específicas según el patrón del código R
combinations_mapping = {
    'checking_account': 'checking_account',
    'savings_account': 'savings_account',
    'savings_account + credit_card': 'credit_card + savings_account',
    'savings_account + insurance': 'insurance + savings_account',
    'checking_account + insurance': 'checking_account + insurance',
    'checking_account + credit_card': 'checking_account + credit_card',
    'checking_account + investment': 'checking_account + investment',
    'savings_account + investment': 'investment + savings_account'
}

# Aplicar el mapeo y dejar OTRA_COMBINACION para el resto
products_final['combinacion_productos'] = products_final['combinacion_productos'].map(
    lambda x: combinations_mapping.get(x, 'OTRA_COMBINACION')
)

# 3. Preparar el dataset de transacciones
# Conteo de transacciones por usuario por categoría
category_counts = (
    transactions
    .groupby('user_id')['merchant_category']
    .value_counts()
    .unstack(fill_value=0)
    .reset_index()
)

# Renombrar columnas para seguir el patrón del código R
category_counts.columns = [
    'user_id' if col == 'user_id' else f'{col}_count' 
    for col in category_counts.columns
]

# Calcular estadísticas adicionales por usuario
transactions_agg = (
    transactions
    .groupby('user_id')
    .agg(
        total_transacciones=('transaction_id', 'count'),
        monto_promedio_transaccion=('amount', 'mean')
    )
    .reset_index()
)

# Unir con los conteos de categorías
transactions_agg = transactions_agg.merge(category_counts, on='user_id', how='left')

# Determinar la categoría favorita basada en conteos
category_cols = [col for col in transactions_agg.columns if col.endswith('_count')]

def get_favorite_category(row):
    categories = {col.replace('_count', ''): row[col] for col in category_cols}
    max_count = max(categories.values())
    # En caso de empate, concatenar con "+"
    fav_categories = [cat for cat, count in categories.items() if count == max_count]
    return " + ".join(fav_categories)

transactions_agg['categoria_favorita'] = transactions_agg.apply(get_favorite_category, axis=1)

# Análisis temporal (mensual)
transactions_monthly = (
    transactions
    .assign(mes=transactions['date'].dt.to_period('M').dt.to_timestamp())
    .groupby(['user_id', 'mes'])
    .agg(
        monto_mes=('amount', 'sum'),
        transacciones_mes=('transaction_id', 'count')
    )
    .reset_index()
)

# Identificar mes con más compras y mayor gasto
def get_monthly_stats(group):
    if group.empty:
        return pd.Series({
            'mes_mas_compras': pd.NaT,
            'mes_mayor_monto': pd.NaT,
            'monto_promedio_mensual': np.nan,
            'transacciones_promedio_mensual': np.nan
        })
    
    idx_max_tx = group['transacciones_mes'].idxmax()
    idx_max_amount = group['monto_mes'].idxmax()
    
    return pd.Series({
        'mes_mas_compras': group.loc[idx_max_tx, 'mes'],
        'mes_mayor_monto': group.loc[idx_max_amount, 'mes'],
        'monto_promedio_mensual': group['monto_mes'].mean(),
        'transacciones_promedio_mensual': group['transacciones_mes'].mean()
    })

transactions_monthly_agg = (
    transactions_monthly
    .sort_values(['user_id', 'mes'])
    .groupby('user_id')
    .apply(get_monthly_stats)
    .reset_index()
)

# Calcular variaciones de un mes al siguiente
def calculate_variations(group):
    if len(group) <= 1:
        return pd.Series({
            'variacion_mensual_promedio': np.nan,
            'variacion_mensual_promedio_pct': np.nan
        })
    
    diffs = group['monto_mes'].diff().dropna().values
    pct_changes = ((group['monto_mes'] / group['monto_mes'].shift(1)) - 1).dropna().values
    
    return pd.Series({
        'variacion_mensual_promedio': np.nanmean(diffs),
        'variacion_mensual_promedio_pct': np.nanmean(pct_changes)
    })

transactions_trend = (
    transactions_monthly
    .sort_values(['user_id', 'mes'])
    .groupby('user_id')
    .apply(calculate_variations)
    .reset_index()
)

# Calcular métricas adicionales
user_agg = (
    transactions
    .groupby('user_id')
    .agg(
        total_spend=('amount', 'sum'),
        n_meses_activos=('date', lambda x: x.dt.to_period('M').nunique()),
        recencia_transaccion=('date', lambda x: (REFERENCE_DATE - x.max()).days)
    )
    .reset_index()
)

# Categoría favorita por monto
cat_monto = (
    transactions
    .groupby(['user_id', 'merchant_category'])
    .agg(total_spend_cat=('amount', 'sum'))
    .reset_index()
)

# Encuentrar categoría con mayor monto
cat_max = (
    cat_monto
    .groupby('user_id')
    .agg(max_spend_cat=('total_spend_cat', 'max'))
    .reset_index()
)

# Determinar categoría favorita por monto
def get_favorite_category_by_amount(group):
    categories = group['merchant_category'].tolist()
    total_spend_fav = group['total_spend_cat'].sum()
    return pd.Series({
        'categoria_favorita_monto': ' + '.join(categories),
        'total_spend_fav': total_spend_fav
    })

cat_fav = (
    cat_monto
    .merge(cat_max, on='user_id')
    .query('total_spend_cat == max_spend_cat')
    .groupby('user_id')
    .apply(get_favorite_category_by_amount)
    .reset_index()
)

# Calcular share de la categoría favorita
cat_fav = (
    cat_fav
    .merge(user_agg[['user_id', 'total_spend']], on='user_id')
    .assign(share_fav=lambda x: x['total_spend_fav'] / x['total_spend'])
)

# Calcular HHI (Herfindahl-Hirschman Index)
def calculate_hhi(group):
    user_spend = group['total_spend_cat'].sum()
    proportions = group['total_spend_cat'] / user_spend
    hhi = (proportions ** 2).sum()
    return pd.Series({'hhi': hhi})

cat_hhi = (
    cat_monto
    .groupby('user_id')
    .apply(calculate_hhi)
    .reset_index()
)

# Unir todas las agregaciones de transacciones
transactions_final = (
    transactions_agg
    .merge(transactions_monthly_agg, on='user_id', how='left')
    .merge(transactions_trend, on='user_id', how='left')
    .merge(user_agg, on='user_id', how='left')
    .merge(cat_fav[['user_id', 'categoria_favorita_monto', 'total_spend_fav', 'share_fav']], 
           on='user_id', how='left')
    .merge(cat_hhi, on='user_id', how='left')
)

# 4. Crear el dataset final uniendo todos los datasets
df_final = (
    demographics
    .merge(products_final, on='user_id', how='left')
    .merge(transactions_final, on='user_id', how='left')
)

# 5. Guardar el dataset
output_path = data_path.parent / 'processed' / 'data_final_jupyter2.csv'
output_path.parent.mkdir(exist_ok=True)
df_final.to_csv(output_path, index=False)

In [118]:
print(df_final.info(memory_usage="deep"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   user_id                         100 non-null    object        
 1   age                             100 non-null    int64         
 2   income_range                    100 non-null    object        
 3   risk_profile                    100 non-null    object        
 4   occupation                      100 non-null    object        
 5   age_range_sturges               100 non-null    category      
 6   primer_producto                 100 non-null    object        
 7   fecha_primer_producto           100 non-null    datetime64[ns]
 8   segundo_producto                55 non-null     object        
 9   fecha_segundo_producto          55 non-null     datetime64[ns]
 10  dias_entre_productos            55 non-null     float64       
 11  antigue

In [119]:
print(df_final.isnull().sum())

user_id                            0
age                                0
income_range                       0
risk_profile                       0
occupation                         0
age_range_sturges                  0
primer_producto                    0
fecha_primer_producto              0
segundo_producto                  45
fecha_segundo_producto            45
dias_entre_productos              45
antiguedad_cliente                 0
checking_account                   0
credit_card                        0
insurance                          0
investment                         0
savings_account                    0
numero_productos                   0
combinacion_productos              0
total_transacciones                0
monto_promedio_transaccion         0
entertainment_count                0
food_count                         0
health_count                       0
shopping_count                     0
supermarket_count                  0
transport_count                    0
t

In [120]:
#Imputación NA

df_final['segundo_producto'] = df_final['segundo_producto'].fillna('none')
df_final['fecha_segundo_producto'] = df_final['fecha_segundo_producto'].fillna('1900-01-01')  # Fecha dummy como marcador
df_final['dias_entre_productos'] = df_final['dias_entre_productos'].fillna(0)

In [121]:
print(df_final.isnull().sum())

user_id                           0
age                               0
income_range                      0
risk_profile                      0
occupation                        0
age_range_sturges                 0
primer_producto                   0
fecha_primer_producto             0
segundo_producto                  0
fecha_segundo_producto            0
dias_entre_productos              0
antiguedad_cliente                0
checking_account                  0
credit_card                       0
insurance                         0
investment                        0
savings_account                   0
numero_productos                  0
combinacion_productos             0
total_transacciones               0
monto_promedio_transaccion        0
entertainment_count               0
food_count                        0
health_count                      0
shopping_count                    0
supermarket_count                 0
transport_count                   0
travel_count                

In [122]:
print(df_final.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   user_id                         100 non-null    object        
 1   age                             100 non-null    int64         
 2   income_range                    100 non-null    object        
 3   risk_profile                    100 non-null    object        
 4   occupation                      100 non-null    object        
 5   age_range_sturges               100 non-null    category      
 6   primer_producto                 100 non-null    object        
 7   fecha_primer_producto           100 non-null    datetime64[ns]
 8   segundo_producto                100 non-null    object        
 9   fecha_segundo_producto          100 non-null    datetime64[ns]
 10  dias_entre_productos            100 non-null    float64       
 11  antigue

In [123]:
# Feature Engineering General - Transformaciones específicas de FE_general.ipynb

# 1. Detectar y convertir columnas con fechas
posibles_fechas = []

for col in df_final.columns:
    if df_final[col].dtype == 'object':
        try:
            pd.to_datetime(df_final[col].dropna().sample(5), errors='raise')
            posibles_fechas.append(col)
        except:
            continue

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

print("Columnas detectadas como fechas:")
print(posibles_fechas)

for col in posibles_fechas:
    df_final[col] = pd.to_datetime(df_final[col], errors='coerce')

# 2. Convertir explícitamente columnas con 'fecha' en su nombre
fecha_cols = [col for col in df_final.columns if "fecha" in col.lower()]
for col in fecha_cols:
    df_final[col] = pd.to_datetime(df_final[col], errors='coerce')

# 3. Eliminar variables específicas y categoria_favorita por que no aplica al modelo
variables_a_eliminar = [
    "total_spend",
    "monto_promedio_mensual",
    "monto_promedio_transaccion",
    "hhi",
    "share_fav",
    "total_transacciones", "categoria_favorita"
]

df = df_final.drop(columns=variables_a_eliminar, errors="ignore")

# Guardar el dataset procesado
output_path = Path('/Users/carloslandaverdealquicirez/Documents/Prometeo_reto/Prometeo_project copy/03.Modelo/data/processed/data_final_fe.csv')
df.to_csv(output_path, index=False)

print(f"Feature Engineering completado. Dataset guardado en: {output_path}")
print(f"Dimensiones finales: {df_final_reduced.shape[0]} filas, {df_final_reduced.shape[1]} columnas")


Columnas detectadas como fechas:
[]
Feature Engineering completado. Dataset guardado en: /Users/carloslandaverdealquicirez/Documents/Prometeo_reto/Prometeo_project copy/03.Modelo/data/processed/data_final_fe.csv
Dimensiones finales: 100 filas, 37 columnas


In [124]:
print(df1.info(memory_usage="deep"))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   user_id                         100 non-null    object        
 1   age                             100 non-null    int64         
 2   income_range                    100 non-null    object        
 3   risk_profile                    100 non-null    object        
 4   occupation                      100 non-null    object        
 5   age_range_sturges               100 non-null    category      
 6   primer_producto                 100 non-null    object        
 7   fecha_primer_producto           100 non-null    datetime64[ns]
 8   segundo_producto                100 non-null    object        
 9   fecha_segundo_producto          100 non-null    datetime64[ns]
 10  dias_entre_productos            100 non-null    float64       
 11  antigue

In [125]:
cols_a_eliminar = [
    "Unnamed: 0", "user_id",
]

fechas = [
    "fecha_primer_producto", "fecha_segundo_producto",
    "mes_mas_compras", "mes_mayor_monto"
]

binarias_explicit = ['checking_account', 'savings_account', 'credit_card', 'investment']


df.drop(columns=[col for col in cols_a_eliminar if col in df.columns], inplace=True)

# Convertir fechas
for col in fechas:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

# Convertir fechas y extraer timestamp  por requerimiento de sklearn
for col in fechas:
    if col in df.columns:
        # Convertir la columna a datetime
        df[col] = pd.to_datetime(df[col], errors='coerce')
        # >>> MODIFICACIÓN: Convertir fecha a formato numérico (timestamp en nanosegundos) usando astype
        df[col + "_ts"] = df[col].astype('int64')
        # Eliminar la columna original si no es necesaria
        df.drop(col, axis=1, inplace=True)

# Aplicar log1p para reducir asimetría de valores altos
if 'total_spend_fav' in df.columns:
    df['total_spend_fav'] = np.log1p(df['total_spend_fav'])

# Aplicar log1p para valores extremos en proporciones
if 'variacion_mensual_promedio_pct' in df.columns:
    df['variacion_mensual_promedio_pct'] = np.log1p(df['variacion_mensual_promedio_pct'])

# Winsorización de variable con outliers importantes
if 'variacion_mensual_promedio' in df.columns:
    p01 = df['variacion_mensual_promedio'].quantile(0.01)
    p99 = df['variacion_mensual_promedio'].quantile(0.99)
    df['variacion_mensual_promedio'] = df['variacion_mensual_promedio'].clip(p01, p99)

# Escalar recencia para estabilidad del gradiente
if 'recencia_transaccion' in df.columns:
    df['recencia_transaccion'] = StandardScaler().fit_transform(df[['recencia_transaccion']])

# Escalar otras variables numéricas continuas
escalar_xgb = [
    'age', 'dias_entre_productos', 'antiguedad_cliente', 'numero_productos',
    'entertainment_count', 'food_count', 'health_count', 'shopping_count',
    'supermarket_count', 'transport_count', 'travel_count'
]
escalar_xgb = [col for col in escalar_xgb if col in df.columns]
df[escalar_xgb] = StandardScaler().fit_transform(df[escalar_xgb])

# Convertir variables binarias explícitas a tipo booleano
for col in binarias_explicit:
    if col in df.columns:
        df[col] = df[col].map({1: True, 0: False}).astype(bool)

# Codificación con LabelEncoder para categóricas (no binarias)
categoricas = [
    'income_range', 'risk_profile', 'occupation', 'age_range_sturges',
    'primer_producto', 'segundo_producto', 'combinacion_productos',
    'categoria_favorita_monto'
]
for col in categoricas:
    if col in df.columns:
        df[col] = LabelEncoder().fit_transform(df[col].astype(str))


columnas_a_quitar = ['fecha_segundo_producto_ts', 'combinacion_productos', 'segundo_producto']

df = df.drop(columns=columnas_a_quitar, errors="ignore")


y = df['insurance']
X = df.drop(columns=['insurance'])

# Guardar datasets preparados para modelado
X.to_csv("../data/processed/X_xgb_reduced_TEST.csv", index=False)
y.to_csv("../data/processed/y_xgb_reduced_TEST.csv", index=False)


In [126]:
print(df.info(memory_usage="deep"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   age                             100 non-null    float64
 1   income_range                    100 non-null    int64  
 2   risk_profile                    100 non-null    int64  
 3   occupation                      100 non-null    int64  
 4   age_range_sturges               100 non-null    int64  
 5   primer_producto                 100 non-null    int64  
 6   dias_entre_productos            100 non-null    float64
 7   antiguedad_cliente              100 non-null    float64
 8   checking_account                100 non-null    bool   
 9   credit_card                     100 non-null    bool   
 10  insurance                       100 non-null    int64  
 11  investment                      100 non-null    bool   
 12  savings_account                 100 n